<a href="https://colab.research.google.com/github/annajli/gaussian_processes/blob/main/Orthogonal_Random_Feature_Map_Approximation_of_RBF_Kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Orthogonal Random Feature Map Approximation of Radial Basis Function Kernels**

#### Author: Anna Li

I provide background for and implement *The Geometry of Random Features* paper by Choromanski, Rowland, et. al (2018) http://mlg.eng.cam.ac.uk/adrian/geometry.pdf

## **1. Introduction**
---

Kernels are useful and ubiquitous but they do not scale well with the size of the dataset. This is because when we perform inference on kernel machines, we often have to compute a kernel matrix whose dimensions depend on the number of training points $N$. A very popular approach to mitigating the scaling performance concern is to approximate the kernel function using **random feature map method**. (Rahimi and Recht, 2007) Namely, we approximate the kernel function by dot products of **random feature maps (RFMs)** that are in a much lower dimension than N. 

$$k(x, x') = \Phi(x)^T\Phi(x') = \langle \Phi(x), \Phi(x') \rangle_{\mathcal{V}} \approx z(x)^Tz(y)$$ 

for $\textbf{z}: \mathbb{R}^d \rightarrow \mathbb{R}^D$ (low-dimensional $D$)

However, RFMs efficiently constructed using **structured matrices** typically has lower accuracy. Recent works show in certain settings, **structured approaches** based on **orthogonal transforms** outpreform **iid methods** in terms of accuracy. (Yu, 2016; Chormanski, 2017)

Orthogonal transforms yield triple win improvements in accuracy, speed, and space complexity but only in extremely specific scenarios which include:

* Gaussian kernel approximation (Yu, 2016)
* dimensionality reduction with **modified Johnson-Lindenstrauss transform**, **angular kernel approximation** (Chrormanski, 2017)
* **cross-polytope LSH** (Andoni, 2015)

This paper proves that **orthogonal random features** for **RBF Kernels** provide strictly better bounds because kernels based on orthogonal random features are characterized by better **spectral propreties** than **unstructured ones**.

### The paper also presents and answers the following key questions for the class of **radial function kernels (RBFs)**:
* How to evaluate gains provided by structured approach (including time for orthogonalization)
* How do gains depend on region of interest and choice of kernel?
* Do **pointwise gains** coming from the orthogonal random features imply downstream application gains (supervised learning tasks)?

### The paper highlights the following contributions:
* Shows asymptotic results:
  * for fixed $||x - y||$ and large dimensional $N$ express the benefit of orthogonality in terms of the *charm function* of the RBF at a given point $x - y$
  * for fixed $N$ and small $||x - y||$: show that under certain conditions, orthogonal random features are superior to iid features for RBFs defined by **bounded 4th moments** of corresponding **Fourier distributions**.

* Show optimality of the random orthogonla feature method for large classes of RBFs under weak conditions

* Provide guarantees that orthogonal random features for RBFs outperforms unstructured ones on downstream tasks such as kernel ridge regression

* Provide empirical benefits from orthogonal features for the following applications:
  * **Pointwise kernel approximation** and **Gram Matrix approximation**
  * **Gaussian Process (GP) Regression**

### Why should I care?
* Their results enable practioners more generally to estimate the benefits from applying orthogonal transforms when pursuing kernel approximation methods.


## **2. Random Fourier Features**
---

**Bochner's Theorem:** If there is an RBF kernel K that is shift-invariant (stationary), then there exists a finite Borel measure $\mu_K \in \mathcal{M}(\mathbb{R^d})$ (the Fourier measure associated with $K$) such that:

$$
K(x, y) = Re\left(\int_{\mathbb{R^d}} \exp( i \langle w, x - y\rangle) \mu_K(dw)\right)
$$

Bochner's theorem allows us to represent a kernel function as an expectation (because we can use the Fourier transform as a probability density) which lets us use **Monte Carlo estimation** for approximating values of RBFs. 

Using the above equation with a standard **Monte Carlo estimation**, we obtain the pointwise kernel estimator:

$$
\hat{K}_{m, d}^{iid}(x, y) = \sum_{i = 1}^{m} \frac{cos(\langle w_i, x - y \rangle)}{m} = \langle \phi_{m, d}(x), \phi_{m, d}(y)\rangle
$$

where $\phi_{m,d}: \mathbb{R}^d \rightarrow \mathbb{R}^{2m}$ is a random feature embedding:

$$
\phi_{m,d}(x) = \left(\frac{1}{\sqrt m} \cos(\langle w_i, x \rangle), \sin(\langle w_i, x \rangle)\right)_{i = 1}^m
$$

for all $x \in \mathbb{R}^d, (w_i)_{i = 1}^m \stackrel{iid}{\sim} \mu_{K}$.

$m$ stands for the total number of random features used. 

Therefore, instead of using the non-linear kernel $K$ on dataset $(x_i)_{i = 1}^N$ we can instead use the linear (inner product) kernel with the randomly embedded dataset $(\phi(x_i))_{i = 1}^N$.

**Lemma 2.1** If $K$ is an RBF, then its Fourier transform $\mu \in \mathcal{M}(\mathbb{R}^n)$ is isotropic: $\mu(A) = \mu(M^{-1}A)$ for all Borel sets $A$, and all $M \in O_n$, the orthogonal group on $\mathbb{R}^d.$

**Def: (Orthogonal Random Features)** Let $K: \mathbb{R}^d x \mathbb{R}^d \rightarrow \mathbb{R}$ be the RBF kenrel with associated **Fourier measure** $\mu_k \in \mathcal{M}(\mathbb{R}^d)$. The **orthogonal random feature map** $\Phi: \mathbb{R}^d \rightarrow \mathbb{R}^{2m} (2m = 2kd, k \in \mathbb{Z})$ associated with $K$ is: 

$$
\phi_{m,d}^{ort}(x) = \left(\frac{1}{\sqrt m} \cos(\langle w_i^l, x \rangle), \sin(\langle w_i^l, x \rangle)\right)_{l = 1, i = 1}^{l = k, i = d}
$$

where blocks of frequency vectors $(w_{i = d}^l)_{l = 1}^k$ are independent and for each frequency vector block, the frequency vectors $w_1^l,...,w_d^l$ are marginally distributed according to $\mu_k$ and are orthogonal almost surely.

**Kernel Estimator:** $$\hat{K}_{m, d}^{ort}(x, y) = \sum_{l = 1}^k \sum_{i = 1}^d \frac{cos(\langle w_i^l, x - y\rangle)}{m} = \langle \phi_{m, d}^{ort}(x), \phi_{m, d}^{ort}(y) \rangle$$

In (Yu, 2016), they present a method called Orthogonal Random Features which addresses both the concern of precision and performance, by enforcing that the rows of the basis matrix $W$ be orthogonal. 

Namely:

$$
W_{ORT} = \frac{1}{\sigma}SQ
$$

where $Q:=$ uniformly distributed orthogonal matrix (set of rows of Q forms a basis in $\mathbb{R}^d$)

$S:=$ diagonal matrix whose diagonal entries are sampled $iid$ from $\chi-$distribution with d degrees of freedom. (S makes the norms of the rows of SQ and G identically distributed)

We must replace G with this decomposition because we need to ensure that the kernel approximator is still unbiased and norms of the matrix still follow a $\chi-$distribution. (Rows of an orthogonal matrix have unit norm)

The paper proves mathematically that $K_{ORF}$ is an unbiased estimator and has lower variance and empirically lower mean squared error (MSE) for the Gaussian kernel on various data sets than $K_{RFF}$.

## **3. Orthogonal Random Features for General RBFs And The Charm Function**

The Yu paper focuses on specifically approximating the Gaussian kernel:

$$
k(x, x') = \exp \left[ \frac{-||x - x'||^2}{2\sigma^2} \right]
$$

but this paper establishes the asymptotic benefits of orthogonal random features for a large class of RBFs and measure its effectiveness using the charm function. 

We introduce the charm function and explain its pivotal role in comparing and contrasting the precision of models based on random feature maps for large $d$. 


**Charm Function:**
The charm function $\Psi_K(z)$ of an RBF Kernel is a function $\mathbb{R}^d \rightarrow \mathbb{R}$ defined at point $z = x - y$ as follows:
 
$$\Psi_K(z) = ||z||^2\frac{d^2\phi^2_K}{dx^2} |_{x = ||z||} ||z||\frac{d\phi^2_K}{dx} |_{x = ||z||}$$

Using the charm function, one can prove that for classes of RBFs defined by positive definite functions $\phi$, that are not parametrized by data dimensionality, charm is always nonnegative across the entire domain in the large dimensionality regime. 

 Large charm values indicate values for where **Mean-Squared Error (MSE)** of the orthogonal estimator is significantly smaller than the MSE of an iid estimator. 

Therefore for large enough $d$, the orthogonal estimator outperforms the iid estimator across the entire domain (with the assumption that tail distributions of corresponding Fourier distributions are not too heavy). 


#### **3.1 The Landscape for Fixed $n$ and Small $||x - y||$**

**Theorem 3.1:** Let $K: \mathbb{R}^d x \mathbb{R}^d$ be an RBF kernel and let $\mu_K \in \mathcal{M}(\mathbb{R}^d)$ be its associated Fourier measure. Suppose bounded fourth moments, then for sufficiently small $||x - y||$, we have:

MSE$(\hat{K}_{m, n}^{iid}(x, y)) >$ MSE$(\hat{K}_{m, n}^{ort}(x, y))$.

Which is satisfied for many classes of RBFs such as Guassian, Matern, and Poisson-Bessel kernels. 

#### **3.2 The Landscape for Fixed $||x - y||$ and Large $n$**

**Def:** Let $M_{\mu_d}(k, d)$ be the $k$-th moment of the random variable $X = ||w||_{2}$ for $w \sim \mu_n$, $\mu_d \in \mathcal{M}(\mathbb{R}^d)$ We say that a sequence of measures ${\mu_d}$ is **concentrated** if:

$$\mathbb{P}[|||w||_2^2 - M_{\mu_d}(2, d)| \geq M_{\mu_d}(2, d)g(d)] \leq \frac{1}{h(d)}$$ for some $g(n) = o_n(1)$ and $h(d) = \omega_n(1)$.

**Assumptions:** Consider a fixed **positive definite**, **shift-invariant** radial basis function $\phi$, a family of RBF kernels $K$, where $K$ on $\mathbb{R}^d x \mathbb{R}^d$, and an associated **concentrated sequence** of Fourier measures (probability densities) $\{\mu_d\}_{d \in \mathbb{N}}$. 

Assume there also exists a constant $C > 0$ and function $\xi: \mathbb{N} \rightarrow \mathbb{R}$ such that $M_{\mu_d}(2k, 2d) \leq (n - 1)(n + 1)...(n + 2k - 3)\xi(k)$ and $\frac{|\xi(k)|}{k!} \leq C^k$ for $k$ large enough. 

**Theorem 3.8** Under the assumptions above, for any fixed $z \in \mathbb{R}_{>0}$ for sufficiently large $n$, $\forall x, y \in \mathbb{R}^d$ such that $||x - y|| = z$,

MSE$(\hat{K}_{m, n}^{iid}(x, y)) >$ MSE$(\hat{K}_{m, n}^{ort}(x, y))$.

#### **3.3 Non-asymptotic Results**

In non-asymptotic regimes, orthogonal random features are still superior to the iid case. We focus on the difference in MSE between iid and orthogonal random features in terms of univariate integrals, which are generally intractable but and can accurately evaluated by deterministic numerical integration. 

**Proposition 3.9** For an RBF kernel $K$ on $\mathbb{R}^d$ with Fourier measure $\mu_K$ and $x, y \in \mathbb{R}^d$, writing $z = x - y$, we have:

MSE$(\hat{K}_{m, n}^{iid}(x, y)) -$ MSE$(\hat{K}_{m, n}^{ort}(x, y)) = \frac{m - 1}{m} \mathbb{E}_{R_1, R_2} \left[ \frac{J_{\frac{n}{2} - 1}(\sqrt{R_1^2 + R_2^2}||z||)\Gamma(\frac{n}{2})}{(\sqrt{R_1^2 + R_2^2}||z||/2)^{\frac{n}{2} - 1}}\right] - \frac{m - 1}{m} \mathbb{E}_{R_1} \left[ \frac{J_{\frac{n}{2} - 1}(R_1||z||)\Gamma(\frac{n}{2})}{(R_1||z||/2)^{\frac{n}{2} - 1}}\right]^2$

where $R_1, R_2$ are independent scalar random variables with the distribution of the norm of a vector drawn from $\mu_K$ and $J_{\alpha}$ is the Bessel function of the first kind of degree $\alpha$.

## **4. Optimality of the Orthogonal Feature Map Mechanism**

We consider unbiased estimators of RBFs introduced in section 2.1. The paper shows that for a significant family of random feature based estimators which are *smooth*, asymptotically for large $n$, the orthogonal estimator is optimal in minimizing mean squared error.

**Def: (Smooth Estimators)** A random feature based estimator $E$ is **smooth** if for a fixed $m, d$ lengths of directions of sampled vectors are chosen independently and furthermore, there exists a function $q: \mathbb{N} \rightarrow \mathbb{R}$ such that $q(x) \rightarrow 0$ as $x \rightarrow \infty$ and for sampled vectors $w_1^{d},...,w_m^d$ the following is true:
$$
\mathbb{E}[|cos(\theta_{i,j}^d)^3|] \leq q(d) * \mathbb{E}[|cos(\theta_{i,j}^d)^2|]
$$

where $\theta_{i,j}^d$ is an angle between $w_i^n$ and $w_j^n$ and $i \neq j$.

Note that state of the art random iid estimators and structured orthogonal estimators are smooth. 

**Theorem 4.2** TO DO FILL IN THIS THEOREM!!!

## **5. Superiority of Orthogonal Random Featres For Downstream Applications**

One key application of random feature maps is **kernel ridge regression (KRR)**. 

**Def:** A matrix $A \in \mathbb{R}^{N x N}$ is a $\Delta$-spectral approximation of another matrix $B \in \mathbb{R}^{NxN}$ for $\Delta \in \mathbb{R_{+}}$ if:

$$
(1 - \Delta)B \preceq A \preceq (1 + \Delta)B
$$

where $X \preceq Y$ stands for $Y - X$ being positive semidefinite.

They show that for $N \in \mathbb{N}$, an RBF kernel $K$ (under assumption Theorem 3.3), an identity matrix $I_N \in mathbb{R}^{N x N}$ and $\lambda > 0$, matrix $\hat{K^{ort}} + \lambda NI_N$ provides a strictly tighter spectral approximation of $K + \lambda N I_N$ than $\hat{K^{iid}} + \lambda NI_N$. Avron (2017) showed that a tighter spectral approximation guarantees a more accurate random feature based kernel approximation. 

## **6. Experiments**

I recreate the empirical results for **Pointwise Kernel, Gram Matrix Estimation, and Gaussian Processes** as presented in the paper.

### Pointwise Kernel and Gram Matrix Estimation

### Gaussian Processes

## **7. Appendix**

---

### **Glossary**
Structured Matrix: A matrix is structured if it admits algorithms for matrix-vector multiplication that have lower complexity than that of a general matrix. 